## Prepare the dataset

In [1]:
import os
import pandas as pd

In [2]:
data_dir = "./"

In [3]:
all_text_files = os.listdir(data_dir)
all_text_files = [files for files in all_text_files if files.endswith(".txt")]
all_text_files

['order_product.txt',
 'order_status_inquiry.txt',
 'payment_method.txt',
 'product_inquiry.txt']

In [4]:
df = pd.DataFrame()
df

""


In [5]:
df.columns

RangeIndex(start=0, stop=0, step=1)

In [6]:
pd.concat([df, pd.DataFrame([{"col1": 1}, {"col1": 2}])])

,col1
0,1
1,2


In [7]:
dataset = pd.DataFrame()
for text_file in all_text_files:
    intent = text_file.split(".")[0]
    print(intent)
    data = {"text": [], "intent": []}
    with open(text_file, "r", encoding="utf-8") as f:
        text_lines = f.readlines()
        for line in text_lines:
            data["text"].append(line.strip())
            data["intent"].append(intent)
    dataset = pd.concat([dataset, pd.DataFrame(data)], ignore_index=True)

order_product
order_status_inquiry
payment_method
product_inquiry


In [8]:
dataset

,text,intent
0,malai 5 ota Redmi ko mobile order garna man xa,order_product
1,malai 1 ta Redmi ko mobile order garna man xa,order_product
2,Apple ko Iphone 5 ota order garna man xa,order_product
3,ma 2 ta Casio ko calculator order garxu,order_product
4,mero order 1 ota Xiaomi ko Redmi mobile ho,order_product
...,...,...
196,Samsung ko mobile kun kun xa,product_inquiry
197,Oppo ra Samsung ko mobile dekhaunu,product_inquiry
198,Xiaomi ko Redmi mobile xa ki xaina,product_inquiry
199,Xiaomi ra Samsung ko mobile lina aako,product_inquiry


## Replacement text

In [9]:
import json

In [10]:
json_file = "./replacement.json"

In [11]:
with open(json_file, "r") as f:
    file_data = json.load(f)

In [12]:
file_data

{'replacement': {'cha': ['chha', 'chhaa', 'xa', 'xha', 'xaa', 'x'],
  'malai': ['mlai', 'malaai', 'malae', 'molai', 'maalai', 'maalaai'],
  'kaha': ['kahaa', 'kah', 'kha', 'ka', 'khaa'],
  'baare': ['bare', 'baaree', 'barey', 'baarey'],
  'maile': ['mailee', 'mailey', 'mailei', 'maileey'],
  'ke': ['k'],
  'paryo': ['paro', 'pariyo', 'pario', 'parryo', 'parryoo'],
  'chahanchu': ['chaahanxu',
   'chahnxu',
   'chaahaanxu',
   'chaahanchu',
   'chaahaanchu'],
  'aaucha': ['auxa', 'aauxa', 'aauchha', 'auchha', 'axa'],
  'baata': ['bata', 'baat'],
  'milcha': ['milxa', 'milchha', 'milx'],
  'sakchu': ['sakxu', 'sakchhu', 'sakchhhu', 'sakchhuu'],
  'pauchu': ['pauxu', 'pauchhu', 'paauchhu', 'paauxu'],
  'laagcha': ['lagcha', 'laagchha', 'lagxa', 'lagx', 'laagxa']}}

In [13]:
replacement_values = list(file_data["replacement"].keys())

In [14]:
replacement_values

['cha',
 'malai',
 'kaha',
 'baare',
 'maile',
 'ke',
 'paryo',
 'chahanchu',
 'aaucha',
 'baata',
 'milcha',
 'sakchu',
 'pauchu',
 'laagcha']

In [15]:
replacement_keys = file_data["replacement"].values()

In [16]:
replacement_keys = [key for item in replacement_keys for key in item]

In [17]:
len(replacement_values), len(replacement_keys)

(14, 59)

In [18]:
a = dict()
a.update({"new_key": "new_value"})
a

{'new_key': 'new_value'}

In [19]:
# reverse mapping for the words
reverse_map = dict()
for key, values in file_data["replacement"].items():
    new_key_value = {v: key for v in values}
    reverse_map.update(new_key_value)

In [20]:
reverse_map

{'chha': 'cha',
 'chhaa': 'cha',
 'xa': 'cha',
 'xha': 'cha',
 'xaa': 'cha',
 'x': 'cha',
 'mlai': 'malai',
 'malaai': 'malai',
 'malae': 'malai',
 'molai': 'malai',
 'maalai': 'malai',
 'maalaai': 'malai',
 'kahaa': 'kaha',
 'kah': 'kaha',
 'kha': 'kaha',
 'ka': 'kaha',
 'khaa': 'kaha',
 'bare': 'baare',
 'baaree': 'baare',
 'barey': 'baare',
 'baarey': 'baare',
 'mailee': 'maile',
 'mailey': 'maile',
 'mailei': 'maile',
 'maileey': 'maile',
 'k': 'ke',
 'paro': 'paryo',
 'pariyo': 'paryo',
 'pario': 'paryo',
 'parryo': 'paryo',
 'parryoo': 'paryo',
 'chaahanxu': 'chahanchu',
 'chahnxu': 'chahanchu',
 'chaahaanxu': 'chahanchu',
 'chaahanchu': 'chahanchu',
 'chaahaanchu': 'chahanchu',
 'auxa': 'aaucha',
 'aauxa': 'aaucha',
 'aauchha': 'aaucha',
 'auchha': 'aaucha',
 'axa': 'aaucha',
 'bata': 'baata',
 'baat': 'baata',
 'milxa': 'milcha',
 'milchha': 'milcha',
 'milx': 'milcha',
 'sakxu': 'sakchu',
 'sakchhu': 'sakchu',
 'sakchhhu': 'sakchu',
 'sakchhuu': 'sakchu',
 'pauxu': 'pauchu',
 

In [21]:
sample_df = pd.DataFrame({
    "text": ["malai saman kinnu chha xa chaahanxu", "k baata payment garne"],
    "intent": ["product_inquiry", "payment_method"]
})

sample_df

,text,intent
0,malai saman kinnu chha xa chaahanxu,product_inquiry
1,k baata payment garne,payment_method


In [22]:
def replace_text(row):
    replaced_text = ""
    for word in row["text"].split():
        if word in replacement_keys:
            print(f"Replaced <{word}> with <{reverse_map[word]}>")
            replaced_text = replaced_text + " " + reverse_map[word]
        else:
            replaced_text = replaced_text + " " + word
    row["text"] = replaced_text
    return row

In [23]:
sample_df.apply(replace_text, axis=1)

Replaced <chha> with <cha>
Replaced <xa> with <cha>
Replaced <chaahanxu> with <chahanchu>
Replaced <k> with <ke>


,text,intent
0,malai saman kinnu cha cha chahanchu,product_inquiry
1,ke baata payment garne,payment_method


In [24]:
intent_dataset = dataset.apply(replace_text, axis=1)

Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <aauxa> with <aaucha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <aauxa> with <aaucha>
Replaced <aauxa> with <aaucha>
Replaced <aauxa> with <aaucha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <k> with <ke>
Replaced <bata> with <baata>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <xa> with <cha>
Replaced <xa> with <cha>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <k> with <ke>
Replaced <k> with <ke

In [25]:
intent_dataset.head(10)

,text,intent
0,malai 5 ota Redmi ko mobile order garna man cha,order_product
1,malai 1 ta Redmi ko mobile order garna man cha,order_product
2,Apple ko Iphone 5 ota order garna man cha,order_product
3,ma 2 ta Casio ko calculator order garxu,order_product
4,mero order 1 ota Xiaomi ko Redmi mobile ho,order_product
5,malai 9 ota Apple ko Iphone mobile Kathmandu ...,order_product
6,"1 ota Sony ko TV, delivery location Dharan",order_product
7,"2 ota mobile, delivery location Sunsari",order_product
8,1 ota calculator delivery location Dharan,order_product
9,3 ta Sony ko TV delivery location Kathmandu,order_product


In [28]:
for i, (original, replaced) in enumerate(zip(dataset.iloc[:10].iterrows(), intent_dataset.iloc[:10].iterrows())):
    print(f"{i}: Original: {original[1]['text']} = Replaced: {replaced[1]['text']}")

0: Original:  malai 5 ota Redmi ko mobile order garna man cha = Replaced:  malai 5 ota Redmi ko mobile order garna man cha
1: Original:  malai 1 ta Redmi ko mobile order garna man cha = Replaced:  malai 1 ta Redmi ko mobile order garna man cha
2: Original:  Apple ko Iphone 5 ota order garna man cha = Replaced:  Apple ko Iphone 5 ota order garna man cha
3: Original:  ma 2 ta Casio ko calculator order garxu = Replaced:  ma 2 ta Casio ko calculator order garxu
4: Original:  mero order 1 ota Xiaomi ko Redmi mobile ho = Replaced:  mero order 1 ota Xiaomi ko Redmi mobile ho
5: Original:  malai 9 ota Apple ko Iphone mobile Kathmandu ma delivery = Replaced:  malai 9 ota Apple ko Iphone mobile Kathmandu ma delivery
6: Original:  1 ota Sony ko TV, delivery location Dharan = Replaced:  1 ota Sony ko TV, delivery location Dharan
7: Original:  2 ota mobile, delivery location Sunsari = Replaced:  2 ota mobile, delivery location Sunsari
8: Original:  1 ota calculator delivery location Dharan = Replac

In [29]:
intent_dataset.to_csv("intent_dataset.csv")

## Load the dataset

In [30]:
df = pd.read_csv("./intent_dataset.csv")

In [31]:
df.sample(2)

,Unnamed: 0,text,intent
29,29,ma product order garna chahanxu,order_product
145,145,pay garne method haru ke ke cha,payment_method
